In [1]:
'''
Notebook for FIG S11G - comparing the effects of muttating synthetic and native genes
'''
# By Kirill Sechkar

# PACKAGE IMPORTS ------------------------------------------------------------------------------------------------------
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
import pickle
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, palettes as bkpalettes, transform as bktransform
from math import pi
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
from contourpy import contour_generator as cgen
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 

# OWN CODE IMPORTS -----------------------------------------------------------------------------------------------------
import synthetic_circuits as circuits
from cell_model import *
from get_steady_state import *
from Fig2.design_guidance_tools import *

/tmp/ipykernel_7179/2133100796.py:25: DeprecationWarning: jax.lib.xla_bridge.get_backend is deprecated; use jax.extend.backend.get_backend.
  print(xla_bridge.get_backend().platform)
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


cpu


Loading BokehJS ...

In [2]:
 # INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, circuit_eff_m_het_div_k_het,  \
    par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.punisher_b_initialise,
    circuits.punisher_b_ode,
    circuits.punisher_b_F_calc,
    circuits.punisher_sep_b_eff_m_het_div_k_het,
    par, init_conds)  # load the circuit

In [3]:
# SPECIFY THE CIRCUIT'S DEFAULT PARAMETERS

# BURDENSOME SYNTHETIC GENE
par['c_b'] = 1
par['a_b'] = 1e5

# PUNISHER
# switch gene conc
par['c_switch'] = 10.0  # gene concentration (nM)
par['a_switch'] = 400.0  # promoter strength (unitless)
par['d_switch']=0.01836
# integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch']/80.0  # RBS weaker than for the switch gene
par['d_int'] = 0.0#0.01836 # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
init_conds['cat_pb'] = 10.0  # gene concentration (nM) - INITIAL CONDITION< NOT PARAMETER as it can be cut out by the integrase
par['a_cat'] = 500.0  # promoter strength (unitless)
# synthetic protease gene
par['c_prot'] = 10.0  # gene concentration (nM)
par['a_prot'] = 25.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0 # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 300.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2 # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['baseline_switch_alt'] = 0
par['p_switch_ac_frac'] = 0.85  # active fraction of protein (i.e. share of molecules bound by the inducer)

# CULTURE MEDIUM
init_conds['s'] = 0.5
par['h_ext'] = 10.5 * (10.0 ** 3)

In [4]:
# DETERMINISTIC SIMULATION PARAMETERS (FOR FINDING GROWTH RATES)
savetimestep = 1  # save time step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver
# simulation time frames
tf = (0,50) # time frame for simulation before burdensome gene loss

In [5]:
# THE CASE OF SYNTHETIC GENE MUTATIONS: FIND STEADY-STATE CELLULAR VARIABLES AND SWITCHING THRESHOLD

# define the range of burdensome gene promoter strengths
a_b_range = np.flip(np.linspace(0,par['a_b'],6)) # note the flip => order from full expression to full mutation

xi_thresholds_smut = np.zeros_like(a_b_range)
xis_smut = np.zeros_like(a_b_range)
l_ss_smut=np.zeros_like(a_b_range)
for i in range(0,len(a_b_range)):
    cur_par = par.copy() # current parameter dictionary
    cur_par['a_b'] = a_b_range[i] # set the burdensome gene promoter strength
    cur_par['k_sxf']=0.0    # not considering punisher action explicitly
    cur_inits = init_conds.copy() # current initial conditions
    
    # get the cellular variables in steady state without burden
    e, F_r, h, xis, chis = values_for_analytical(cur_par, ode_with_circuit, cur_inits,
                                                 circuit_genes, circuit_miscs,
                                                 circuit_name2pos,
                                                 circuit_F_calc,
                                                 circuit_eff_m_het_div_k_het)
    # record the cellular variables
    cellvars = {'e': e, 'F_r': F_r, # translation elongation rate and ribosome trnscription regulation
                'h': h, # intacellular chlorampenicol concentration
                # burden values
                'xi_a': xis['a'], 'xi_r': xis['r'], 'xi_other_genes': xis['other'], 'xi_cat': xis['cat'],
                'xi_switch_max': xis['switch (max)'], 'xi_int_max': xis['int (max)'], 'xi_prot': xis['prot'],
                # protein degradation correction factors for the switch protein and the integrase
                'chi_switch': chis['switch'], 'chi_int': chis['int']}
    # the burden value xi with the synthetic gene
    xis_smut[i] = xis['a'] + xis['r'] + xis['cat'] + xis['prot'] + xis['other']
    
    # get the threshold burden value for the punisher's switching
    _, xi_thresholds_smut[i], _, _, _ = threshold_mfchanges(cur_par, cellvars)
    
    # get the steady-state growth rate
    sol = ode_sim(cur_par,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(cur_inits, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(cur_par, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
    ts = np.array(sol.ts)
    xs = np.array(sol.ys)
    # record values
    _, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, cur_par, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                           circuit_eff_m_het_div_k_het)
    l_ss_smut[i]=np.float64(ls[-1])

print(xi_thresholds_smut)
print(xis_smut)
print(l_ss_smut)


[152170.82971196 152170.82971196 152170.82971196 152170.82971196
 152170.82971196 152170.82971196]
[170038.70295772 165763.21868979 161487.73442185 157212.25015392
 152936.76588599 148661.28161806]
[1.33642653 1.37091937 1.40723897 1.44553306 1.48596164 1.52511943]


In [6]:
# THE CASE OF SYNTHETIC GENE MUTATIONS: FIND BURDEN MARGINS AND GROWTH BENEFTIS

Dxis_smut = np.zeros_like(a_b_range)
gbs_smut = np.zeros_like(a_b_range)
for i in range(0,len(a_b_range)):
    # find the burden margin
    Dxis_smut[i] = xis_smut[i] - xi_thresholds_smut[i]
    # find the growth advantage (note: the first value, with full expression, is the reference)
    gbs_smut[i] = l_ss_smut[i]/l_ss_smut[0]-1
    
print(Dxis_smut)
print(gbs_smut)

# get values for the cases where pounishment is not triggered
Dxis_smut_nopun = Dxis_smut#[Dxis_smut >= 0]
gbs_smut_nopun = gbs_smut#[Dxis_smut >= 0]

[17867.87324576 13592.38897783  9316.90470989  5041.42044196
   765.93617403 -3509.5480939 ]
[0.         0.02580976 0.05298641 0.0816405  0.11189176 0.14119213]


In [7]:
# THE CASE OF NATIVE GENE MUTATIONS: FIND STEADY-STATE CELLULAR VARIABLES AND SWITCHING THRESHOLD
s_range = np.linspace(init_conds['s'],0.65,6)

xi_thresholds_nmut = np.zeros_like(s_range)
xis_nmut = np.zeros_like(s_range)
l_ss_nmut=np.zeros_like(s_range)
for i in range(0,len(a_b_range)):
    cur_par = par.copy() # current parameter dictionary
    cur_par['k_sxf']=0.0    # not considering punisher action explicitly
    cur_inits = init_conds.copy() # current initial conditions
    cur_inits['s'] = s_range[i] # set the nutrient concentration
    
    # get the cellular variables in steady state without burden
    e, F_r, h, xis, chis = values_for_analytical(cur_par, ode_with_circuit, cur_inits,
                                                 circuit_genes, circuit_miscs,
                                                 circuit_name2pos,
                                                 circuit_F_calc,
                                                 circuit_eff_m_het_div_k_het)
    # record the cellular variables
    cellvars = {'e': e, 'F_r': F_r, # translation elongation rate and ribosome trnscription regulation
                'h': h, # intacellular chlorampenicol concentration
                # burden values
                'xi_a': xis['a'], 'xi_r': xis['r'], 'xi_other_genes': xis['other'], 'xi_cat': xis['cat'],
                'xi_switch_max': xis['switch (max)'], 'xi_int_max': xis['int (max)'], 'xi_prot': xis['prot'],
                # protein degradation correction factors for the switch protein and the integrase
                'chi_switch': chis['switch'], 'chi_int': chis['int']}
    # the burden value xi with the synthetic gene
    xis_nmut[i] = xis['a'] + xis['r'] + xis['cat'] + xis['prot'] + xis['other']
    
    # get the threshold burden value for the punisher's switching
    _, xi_thresholds_nmut[i], _, _, _ = threshold_mfchanges(cur_par, cellvars)
    
    # get the steady-state growth rate
    sol = ode_sim(cur_par,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(cur_inits, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(cur_par, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
    ts = np.array(sol.ts)
    xs = np.array(sol.ys)
    # record values
    _, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, cur_par, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                           circuit_eff_m_het_div_k_het)
    l_ss_nmut[i]=np.float64(ls[-1])
    
print(xi_thresholds_nmut)
print(xis_nmut)
print(l_ss_nmut)

[152170.82971196 159276.19999795 166289.64212126 173213.15065441
 180048.41614766 186796.91568198]
[170038.70295772 172478.51199329 175014.52475349 177634.55012237
 180328.36536718 183087.33475809]
[1.33642653 1.38961316 1.44065802 1.48966989 1.53674789 1.57892814]


In [8]:
# THE CASE OF NATIVE GENE MUTATIONS: FIND BURDEN MARGINS AND GROWTH BENEFTIS

Dxis_nmut = np.zeros_like(s_range)
gbs_nmut = np.zeros_like(s_range)
for i in range(0,len(s_range)):
    # find the burden margin
    Dxis_nmut[i] = xis_nmut[i] - xi_thresholds_nmut[i]
    # find the growth advantage (note: the first value, with original - bad - culture medium nutrient qualities, is the reference)
    gbs_nmut[i] = l_ss_nmut[i]/l_ss_nmut[0]-1
    
# get values for the cases where pounishment is not triggered
Dxis_nmut_nopun = Dxis_nmut#[Dxis_nmut >= 0]
gbs_nmut_nopun = gbs_nmut#[Dxis_nmut >= 0]

In [11]:
# PLOT

# set ranges for the plot
gb_plot_range=(0, 0.2)
Dxi_plot_range=(-5000, 20000)

# set up the figure
gbdxi_fig = bkplot.figure(
    frame_width=240,
    frame_height=180,
    x_axis_label="Δλ (growth advantage)",
    y_axis_label="ΔΞ (burden margin)",
    x_range=gb_plot_range,
    y_range=Dxi_plot_range,
    tools="box_zoom,pan,hover,reset,save"
)
gbdxi_fig.xaxis.axis_label_text_font_size = '9pt'
gbdxi_fig.yaxis.axis_label_text_font_size = '9pt'
gbdxi_fig.output_backend = 'svg'
# add shading for where punishment is triggered
gbdxi_fig.add_layout(
    bkmodels.PolyAnnotation(xs=[gb_plot_range[0], gb_plot_range[0], gb_plot_range[1], gb_plot_range[1]],
                            ys=[Dxi_plot_range[0], 0, 0, Dxi_plot_range[0]],
                            fill_color='gray', fill_alpha=0.5, hatch_pattern='horizontal_wave',
                            line_color=None)
)
gbdxi_fig.add_layout(bkmodels.Label(x=gb_plot_range[0], y=Dxi_plot_range[0],
                                x_offset=8, y_offset=8,
                                text='Punisher triggered',
                                text_font_size='10pt',
                                    background_fill_color='white', background_fill_alpha=1))

# plot the case of native gene mutations
gbdxi_fig.line(gbs_nmut_nopun, Dxis_nmut_nopun,
               line_width=2,
               color='blue', legend_label='Native genes')
gbdxi_fig.scatter(gbs_nmut_nopun, Dxis_nmut_nopun,
                  marker='circle', line_width=2,
                  color='blue', legend_label='Native genes')
# plot the case of synthetic gene mutations
gbdxi_fig.line(gbs_smut_nopun, Dxis_smut_nopun,
               line_width=2,
               color='red', legend_label='Synthetic gene')
gbdxi_fig.scatter(gbs_smut_nopun, Dxis_smut_nopun,
                  marker='circle', line_width=2,
                  color='red', legend_label='Synthetic gene')

# set up the legend
gbdxi_fig.legend.location = 'top_right'
gbdxi_fig.legend.label_text_font_size = '8pt'
gbdxi_fig.legend.click_policy = 'hide'
gbdxi_fig.legend.title = 'Mutation:'
gbdxi_fig.legend.title_text_font_size = '8pt'
gbdxi_fig.legend.margin = 5
gbdxi_fig.legend.spacing = 5
gbdxi_fig.legend.padding = 5

# show the plot
bkplot.show(gbdxi_fig)

In [10]:
print(s_range)

[0.5  0.53 0.56 0.59 0.62 0.65]
